In [ ]:
# !pip install plotly
# !pip install dash
# !pip install jupyter-dash

In [ ]:
import pandas as pd
import numpy as np
import dash
import plotly.express as px
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
import plotly.figure_factory as ff
import datetime as dt
import pandas_profiling

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  


In [ ]:
df = pd.read_csv('/content/netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [ ]:
# general information about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [ ]:
# Null values in each column
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [ ]:
# the percentage of null values for each column
def missing_values(n):
    df1=pd.DataFrame()
    df1["missing_values, %"]=df.isnull().sum()*100/len(df.isnull())
    df1["missing_values, sum"]=df.isnull().sum()
    return df1.sort_values(by="missing_values, %", ascending=False)
missing_values(df)

,"missing_values, %","missing_values, sum"
director,29.908028,2634
country,9.435676,831
cast,9.367549,825
date_added,0.113546,10
rating,0.045418,4
duration,0.034064,3
show_id,0.000000,0
type,0.000000,0
title,0.000000,0
release_year,0.000000,0


## Data Cleaning

In [ ]:
# convert date_added to pandas datetime type 
df['date_added'] = pd.to_datetime(df['date_added'])

In [ ]:
# fill null values in director, cast and country columns to be set to 'unknown', and for duration to be 'Not added'
df["director"]=df["director"].fillna("Unknown")
df["cast"]=df["cast"].fillna("Unknown")
df["country"]=df["country"].fillna("Unknown")
df[['duration']].replace(np.nan,'Not Added')

,duration
0,90 min
1,2 Seasons
2,1 Season
3,1 Season
4,2 Seasons
...,...
8802,158 min
8803,2 Seasons
8804,88 min
8805,88 min


In [ ]:
# check if your data still has null values
df.isnull().sum()

show_id            0
type               0
title              0
director           0
cast               0
country            0
date_added        10
release_year       0
rating             4
duration        2679
listed_in          0
description        0
year_added        10
month_added       10
month_name        10
season_count    6131
dtype: int64

In [ ]:
# fill the rest of null values to previous and next values
df = df.ffill().bfill()
df.isnull().sum()

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
year_added      0
month_added     0
month_name      0
season_count    0
dtype: int64

## Feature Engineering

In [ ]:
# Create features year_added, month_added, month_name, duration: the value in duration column that doesn't contain the word season
# season_count: in duration column values that contain the word season
df['year_added'] = df['date_added'].dt.year.astype(int)
df['month_added'] = df['date_added'].dt.month
df['month_name'] = df['date_added'].dt.month_name()
df['season_count'] = df.apply(lambda x : x['duration'].split(" ")[0] if "Season" in str(x['duration']) else "", axis = 1)
df['duration'] = df.apply(lambda x : str(x['duration']).split(" ")[0] if "Season" not in str(x['duration']) else "", axis = 1)

In [ ]:
# convert all columns to its correct format (e.g.: numeric columns saved as object to float)
cols = ['duration', 'release_year', 'season_count']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

# Data Analysis

In [ ]:
# How many TV-Shows and Movies in this data?
df["type"].value_counts()

Movie      6131
TV Show    2676
Name: type, dtype: int64

In [ ]:
# How many Horror Movies and Tv Shows on Netflix?
print('no. of Horror movies on Netflix: ',((df["listed_in"].str.contains("Horror"))[(df["type"]=="Movie")]).sum())
print('no. of Horror movies on TV shows: ', ((df["listed_in"].str.contains("Horror"))[(df["type"]=="TV Show")]).sum())

no. of Horror movies on Netflix:  357
no. of Horror movies on TV shows:  75


In [ ]:
# What are the top 10 horror movies?
mask = df["listed_in"].str.contains("Horror")
df[mask].groupby("title")[['type', 'title', 'country','description']].sum().head(10)

,type,title,country,description
title,,,,
#Alive,Movie,#Alive,South Korea,"As a grisly virus rampages a city, a lone man ..."
122,Movie,122,Egypt,"After an awful accident, a couple admitted to ..."
13 Cameras,Movie,13 Cameras,United States,Young parents-to-be Claire and Ryan move into ...
13 Sins,Movie,13 Sins,United States,A man agrees to appear on a game show with a $...
14 Cameras,Movie,14 Cameras,United States,"Upping the “13 Cameras” ante, this sequel find..."
1920,Movie,1920,India,An architect and his wife move into a castle t...
1BR,Movie,1BR,United States,"Seeking her independence, a young woman moves ..."
1st Summoning,Movie,1st Summoning,United States,Student filmmakers uncover occult rituals tied...
23:59,Movie,23:59,Singapore,"When an army recruit is found dead, his fellow..."


In [ ]:
# What is the average duration of the movie?
sort= df["duration"]!=0
df[sort]["duration"].median()

98.0

In [ ]:
# Create a summary statstics table for duration and season count features using dedicated pandas function
df[['season_count', 'duration']].describe()

,season_count,duration
count,0.0,8807.000000
mean,NaN,98.363802
std,NaN,28.149612
min,NaN,3.000000
25%,NaN,86.000000
50%,NaN,98.000000
75%,NaN,113.000000
max,NaN,312.000000


In [ ]:
# longest movie ever
df[df["duration"]== max(df["duration"])]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added,month_name,season_count
4253,s4254,Movie,Black Mirror: Bandersnatch,Unknown,"Fionn Whitehead, Will Poulter, Craig Parkinson...",United States,2018-12-28,2018,TV-MA,312.0,"Dramas, International Movies, Sci-Fi & Fantasy","In 1984, a young programmer begins to question...",2018,12,December,NaN
4254,s4255,TV Show,Hashoter Hatov,Unknown,"Moshe Ivgy, Guy Loel, Yuval Semo, Yigal Adika,...",Israel,2018-12-28,2015,TV-MA,312.0,"Crime TV Shows, International TV Shows, TV Com...",An honest – though overzealous – police office...,2018,12,December,NaN


In [ ]:
# top 10 countries in Movie releases
df[df["type"]=="Movie"].groupby("country")[["show_id"]].count().sort_values(by="show_id", ascending=False).head(10)


,show_id
country,
United States,2058
India,893
Unknown,440
United Kingdom,206
Canada,122
Spain,97
Egypt,92
Nigeria,86
Indonesia,77


In [ ]:
# top 10 countries in Tv-Show releases 
df[df["type"]=="TV Show"].groupby("country")[["show_id"]].count().sort_values(by="show_id", ascending=False).head(10)


,show_id
country,
United States,760
Unknown,391
United Kingdom,213
Japan,169
South Korea,158
India,79
Taiwan,68
Canada,59
France,49


In [ ]:
# How many movies released in 2018 for each category?
types_2018 = df[df['release_year'] == 2018]['listed_in'].reset_index()
types_2018 = pd.DataFrame(types_2018.listed_in.str.split(expand=True))[0].replace(",|\'",'',regex=True).reset_index()
types_2018.groupby(0).count()

,index
0,
Action,81
Anime,28
British,37
Children,61
Classic,2
Comedies,138
Crime,71
Cult,1
Documentaries,117


In [ ]:
# What are movies for Tom Cruise
df[df["cast"].str.contains("Tom Cruise")]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added,month_name,season_count
341,s342,Movie,Magnolia,Paul Thomas Anderson,"John C. Reilly, Philip Baker Hall, Tom Cruise,...",United States,2021-08-01,1999,R,189.0,"Dramas, Independent Movies","Through chance, history and divine interventio...",2021,8,August,NaN
1254,s1255,Movie,Rain Man,Barry Levinson,"Dustin Hoffman, Tom Cruise, Valeria Golino, Ge...",United States,2021-03-01,1988,R,134.0,"Classic Movies, Dramas","Motivated by money, a selfish workaholic seeki...",2021,3,March,NaN


### Visualizations

In [ ]:
# Content Type on Netflix
categ = df['type'].value_counts().to_frame().reset_index().rename(columns={'index':'type', 'type':'count'})
fig = go.Figure([go.Pie(labels=categ['type'], values=categ['count'], pull=[0.05, 0], marker=dict(colors=["#6ad49b", "#a678de"]))])
fig = fig.update_traces(hoverinfo='label+percent', textinfo='value+percent', textfont_size=15, insidetextorientation='radial')
fig = fig.update_layout(title='Types of Content on Netflix', title_x=0.5)
fig.show()

In [ ]:
r = df['rating'].value_counts()[:15].to_dict()
data = dict(
    count = r.values(),
    rating = r.keys())
fig = px.funnel(data, x='count', y='rating')
fig.update_layout(title='Count of Content Ratings on Netflix', title_x=0.5)
fig.show()

In [ ]:
# Growth in content over the years¶
d1 = df[df["type"] == "TV Show"]
d2 = df[df["type"] == "Movie"]

col = "year_added"

vc1 = d1[col].value_counts().reset_index()
vc1 = vc1.rename(columns = {col : "count", "index" : col})
vc1['percent'] = vc1['count'].apply(lambda x : 100*x/sum(vc1['count']))
vc1 = vc1.sort_values(col)

vc2 = d2[col].value_counts().reset_index()
vc2 = vc2.rename(columns = {col : "count", "index" : col})
vc2['percent'] = vc2['count'].apply(lambda x : 100*x/sum(vc2['count']))
vc2 = vc2.sort_values(col)

trace1 = go.Scatter(x=vc1[col], y=vc1["count"], name="TV Shows", marker=dict(color="#a678de"))
trace2 = go.Scatter(x=vc2[col], y=vc2["count"], name="Movies", marker=dict(color="#6ad49b"))
data = [trace1, trace2]
layout = go.Layout(title="Content added over the years", legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

In [ ]:
col = 'month_added'
vc1 = df[col].value_counts().reset_index()
vc1 = vc1.rename(columns = {col : "count", "index" : col})
vc1['percent'] = vc1['count'].apply(lambda x : 100*x/sum(vc1['count']))
vc1 = vc1.sort_values(col)

trace1 = go.Bar(x=vc1[col], y=vc1["count"], name="TV Shows", marker=dict(color="#a678de"))
data = [trace1]
layout = go.Layout(title="In which month, the conent is added the most?", legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

In [ ]:
# The ratings of the content
col = "rating"
vc1 = d1[col].value_counts().reset_index()
vc1 = vc1.rename(columns = {col : "count", "index" : col})
vc1['percent'] = vc1['count'].apply(lambda x : 100*x/sum(vc1['count']))
vc1 = vc1.sort_values(col)

vc2 = d2[col].value_counts().reset_index()
vc2 = vc2.rename(columns = {col : "count", "index" : col})
vc2['percent'] = vc2['count'].apply(lambda x : 100*x/sum(vc2['count']))
vc2 = vc2.sort_values(col)

trace1 = go.Bar(x=vc1[col], y=vc1["count"], name="TV Shows", marker=dict(color="#a678de"))
trace2 = go.Bar(x=vc2[col], y=vc2["count"], name="Movies", marker=dict(color="#6ad49b"))
data = [trace1, trace2]
layout = go.Layout(title="Content added over the years", legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

In [ ]:
# What are the top Categories?
from collections import Counter
col = "listed_in"
categories = ", ".join(d2['listed_in']).split(", ")
counter_list = Counter(categories).most_common(50)
labels = [_[0] for _ in counter_list][::-1]
values = [_[1] for _ in counter_list][::-1]
trace1 = go.Bar(y=labels, x=values, orientation="h", name="TV Shows", marker=dict(color="#a678de"))

data = [trace1]
layout = go.Layout(title="Content added over the years", legend=dict(x=0.1, y=1.1, orientation="h"))
content_added = go.Figure(data, layout=layout)
content_added.show()

In [ ]:
small = df[df["type"] == "Movie"]
small = small[small["country"] == "United States"]

col = "director"
categories = ", ".join(small[col].fillna("")).split(", ")
counter_list = Counter(categories).most_common(12)
counter_list = [_ for _ in counter_list if _[0] != ""]
labels = [_[0] for _ in counter_list][::-1]
values = [_[1] for _ in counter_list][::-1]
trace1 = go.Bar(y=labels, x=values, orientation="h", name="TV Shows")

data = [trace1]
layout = go.Layout(title="Movie Directors from US with most content", legend=dict(x=0.1, y=1.1, orientation="h"))
movie_director = go.Figure(data, layout=layout)
movie_director.show()

In [ ]:
df.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added,month_name,season_count
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,2021-09-25,2020,PG-13,90.0,Documentaries,"As her father nears the end of his life, filmm...",2021,9,September,NaN


In [ ]:
app=JupyterDash(__name__)
server = app.server
drop_down = df['year_added'].unique().tolist()

app.layout = html.Div(children=[
    html.H1(children='Netflix Dash'),
    html.Div(children='''
        Netflix Report.
    '''),
    html.Div([
    html.Div([
        html.H3('Select year:')
    ], 
    ),

    dcc.Dropdown(
    className="state-dropdown",
    id='year-select', 
    options=[{'label': year, 'value': year} for year in drop_down],
    value=drop_down[0],
    clearable=False,
    )], className='dropdown'),
    html.Div([
        dcc.Graph(
            id='graph1',
            figure=fig
        ),
    ]),
    html.Div([
        dcc.Graph(
            id='graph2',
            figure=fig
        ),
    ]),
    html.Div([
      dcc.Graph(
          id='graph3',
          figure=fig
      ),
    ]),
])
@app.callback(
    Output('graph2', 'figure'),
    [Input('year-select', 'value')])

def update_graph(year_selected):
    df_subset = df[df['year_added'] == year_selected]
    fig = px.pie(
    data_frame=df_subset,
    names='type',
    hole=0.8,
    title='TV Show vs. Movie')
    return fig

@app.callback(
    Output('graph1', 'figure'),
    [Input('year-select', 'value')])

def update_graph(year_selected):
    df_subset = df[df['year_added'] == year_selected]
    r = df_subset['rating'].value_counts()[:15].to_dict()
    data = dict(count = r.values(),
    rating = r.keys())
    fig = px.funnel(data, x='count', y='rating')
    fig.update_layout(title='Count of Content Ratings on Netflix', title_x=0.5)
    return fig
#3
@app.callback(
    Output('graph3', 'figure'),
    [Input('year-select', 'value')])

def update_graph(year_selected): 
    col = "listed_in"
    df_subset = df[df['year_added'] == year_selected]
    categories = ", ".join(df_subset['listed_in']).split(", ")
    counter_list = Counter(categories).most_common(50)
    labels = [_[0] for _ in counter_list][::-1]
    values = [_[1] for _ in counter_list][::-1]
    trace1 = go.Bar(y=labels, x=values, orientation="h", name="TV Shows", marker=dict(color="#a678de"))

    data = [trace1]
    layout = go.Layout(title="Content added over the years", legend=dict(x=0.1, y=1.1, orientation="h"))
    fig = go.Figure(data, layout=layout)
    return fig

app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>